### Import Required Libraries and Set Up Environment Variables

In [20]:
!pip install python-dotenv

In [159]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')
print(NASA_API_KEY)

X5Rsa8m3mUgQ9w4PcCkoycbNsfumMRINc4SatjFF


### CME Data

In [160]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for CME
params = {
    "startDate": startDate,
    "endDate": end_date,
    "api_key": NASA_API_KEY
}

query_url = f"{base_url}CME?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"
print(query_url)

https://api.nasa.gov/DONKI/CME?startDate=2013-05-01&endDate=2024-05-01&api_key=X5Rsa8m3mUgQ9w4PcCkoycbNsfumMRINc4SatjFF


In [161]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url, params=params)

In [162]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [163]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
formatted_json = json.dumps(cme_json, indent=4)
#formatted_json

In [105]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]

# Display the DataFrame
cme_df

,activityID,startTime,linkedEvents
0,2024-08-18T20:00:00-CME-001,2024-08-18T20:00Z,None
1,2024-08-19T09:12:00-CME-001,2024-08-19T09:12Z,None
2,2024-08-19T12:36:00-CME-001,2024-08-19T12:36Z,None
3,2024-08-19T13:23:00-CME-001,2024-08-19T13:23Z,None
4,2024-08-19T16:24:00-CME-001,2024-08-19T16:24Z,None
...,...,...,...
104,2024-09-14T15:36:00-CME-001,2024-09-14T15:36Z,[{'activityID': '2024-09-14T15:13:00-FLR-001'}...
105,2024-09-14T18:00:00-CME-001,2024-09-14T18:00Z,None
106,2024-09-14T18:12:00-CME-001,2024-09-14T18:12Z,None
107,2024-09-14T19:36:00-CME-001,2024-09-14T19:36Z,None


In [106]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])
cme_df.head()

,activityID,startTime,linkedEvents
23,2024-08-22T10:00:00-CME-001,2024-08-22T10:00Z,[{'activityID': '2024-08-22T08:04:00-FLR-001'}]
25,2024-08-23T02:00:00-CME-001,2024-08-23T02:00Z,[{'activityID': '2024-08-23T01:05:00-FLR-001'}...
43,2024-08-30T12:53:00-CME-001,2024-08-30T12:53Z,[{'activityID': '2024-08-30T12:13:00-FLR-001'}]
49,2024-09-01T03:24:00-CME-001,2024-09-01T03:24Z,[{'activityID': '2024-09-04T09:40:00-IPS-001'}...
50,2024-09-01T12:23:00-CME-001,2024-09-01T12:23Z,[{'activityID': '2024-09-01T11:45:00-FLR-001'}...


In [107]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_df.index:
    # Get the corresponding value from row i in 'activityID'
    activity_id = cme_df.at[i, 'activityID']
    # Get the corresponding value from row i in 'startTime'
    start_time = cme_df.at[i, 'startTime']
    # Get the list of dictionaries from row i in 'linkedEvents'
    linked_events = cme_df.at[i, 'linkedEvents']
    
    # Check if linked_events is not empty and is a list
    if linked_events and isinstance(linked_events, list):
        # Iterate over each dictionary in the list
        for event in linked_events:
            # Append a new dictionary to the expanded_rows list
            expanded_rows.append({
                'activityID': activity_id,
                'startTime': start_time,
                'linkedEvent': event
            })
expanded_df = pd.DataFrame(expanded_rows)
expanded_df.head()

,activityID,startTime,linkedEvent
0,2024-08-22T10:00:00-CME-001,2024-08-22T10:00Z,{'activityID': '2024-08-22T08:04:00-FLR-001'}
1,2024-08-23T02:00:00-CME-001,2024-08-23T02:00Z,{'activityID': '2024-08-23T01:05:00-FLR-001'}
2,2024-08-23T02:00:00-CME-001,2024-08-23T02:00Z,{'activityID': '2024-08-26T17:00:00-IPS-001'}
3,2024-08-23T02:00:00-CME-001,2024-08-23T02:00Z,{'activityID': '2024-08-27T07:33:00-IPS-001'}
4,2024-08-23T02:00:00-CME-001,2024-08-23T02:00Z,{'activityID': '2024-08-28T00:00:00-GST-001'}


In [108]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

# Log the error or print it for debugging
def extract_activityID_from_dict(input_dict):
    try:
        if isinstance(input_dict, dict):
            return input_dict.get('activityID', 'No activityID found')
        else:
            raise TypeError("Input is not a dictionary")
    except (ValueError, TypeError) as e:
        print(f"Error extracting activityID: {e}")
        return None

example_linked_events = expanded_df.loc[0, 'linkedEvent']
extracted_id = extract_activityID_from_dict(example_linked_events)
print(f"Extracted activityID: {extracted_id}")

Extracted activityID: 2024-08-22T08:04:00-FLR-001


In [110]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_df['GST_ActivityID'] = cme_df['linkedEvents'].apply(
    lambda linked_events: [extract_activityID_from_dict(event) for event in linked_events]
)
cme_df.head()

/var/folders/mp/94lfj_vx2ss5ynv_8hvxkgb40000gn/T/ipykernel_33968/838688080.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cme_df['GST_ActivityID'] = cme_df['linkedEvents'].apply(


,activityID,startTime,linkedEvents,GST_ActivityID
23,2024-08-22T10:00:00-CME-001,2024-08-22T10:00Z,[{'activityID': '2024-08-22T08:04:00-FLR-001'}],[2024-08-22T08:04:00-FLR-001]
25,2024-08-23T02:00:00-CME-001,2024-08-23T02:00Z,[{'activityID': '2024-08-23T01:05:00-FLR-001'}...,"[2024-08-23T01:05:00-FLR-001, 2024-08-26T17:00..."
43,2024-08-30T12:53:00-CME-001,2024-08-30T12:53Z,[{'activityID': '2024-08-30T12:13:00-FLR-001'}],[2024-08-30T12:13:00-FLR-001]
49,2024-09-01T03:24:00-CME-001,2024-09-01T03:24Z,[{'activityID': '2024-09-04T09:40:00-IPS-001'}...,"[2024-09-04T09:40:00-IPS-001, 2024-09-04T13:17..."
50,2024-09-01T12:23:00-CME-001,2024-09-01T12:23Z,[{'activityID': '2024-09-01T11:45:00-FLR-001'}...,"[2024-09-01T11:45:00-FLR-001, 2024-09-01T23:14..."


In [111]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_df = cme_df.dropna(subset=['GST_ActivityID'])
cme_df.head()

,activityID,startTime,linkedEvents,GST_ActivityID
23,2024-08-22T10:00:00-CME-001,2024-08-22T10:00Z,[{'activityID': '2024-08-22T08:04:00-FLR-001'}],[2024-08-22T08:04:00-FLR-001]
25,2024-08-23T02:00:00-CME-001,2024-08-23T02:00Z,[{'activityID': '2024-08-23T01:05:00-FLR-001'}...,"[2024-08-23T01:05:00-FLR-001, 2024-08-26T17:00..."
43,2024-08-30T12:53:00-CME-001,2024-08-30T12:53Z,[{'activityID': '2024-08-30T12:13:00-FLR-001'}],[2024-08-30T12:13:00-FLR-001]
49,2024-09-01T03:24:00-CME-001,2024-09-01T03:24Z,[{'activityID': '2024-09-04T09:40:00-IPS-001'}...,"[2024-09-04T09:40:00-IPS-001, 2024-09-04T13:17..."
50,2024-09-01T12:23:00-CME-001,2024-09-01T12:23Z,[{'activityID': '2024-09-01T11:45:00-FLR-001'}...,"[2024-09-01T11:45:00-FLR-001, 2024-09-01T23:14..."


In [112]:
# print out the datatype of each column in this DataFrame:
cme_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 23 to 104
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      21 non-null     object
 1   startTime       21 non-null     object
 2   linkedEvents    21 non-null     object
 3   GST_ActivityID  21 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [113]:
# Convert the 'GST_ActivityID' column to string format
cme_df['GST_ActivityID'] = cme_df['GST_ActivityID'].apply(lambda x: [str(id) for id in x])

# Convert 'startTime' to datetime format
cme_df['startTime'] = pd.to_datetime(cme_df['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
cme_df = cme_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'})

# Drop 'linkedEvents' column
cme_df = cme_df.drop(columns=['linkedEvents'])
cme_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 23 to 104
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           21 non-null     object             
 1   startTime_CME   21 non-null     datetime64[ns, UTC]
 2   GST_ActivityID  21 non-null     object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 1.2+ KB


In [114]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme_df = cme_df[cme_df['GST_ActivityID'].apply(lambda ids: any('GST' in str(id) for id in ids))]
cme_df

,cmeID,startTime_CME,GST_ActivityID
25,2024-08-23T02:00:00-CME-001,2024-08-23 02:00:00+00:00,"[2024-08-23T01:05:00-FLR-001, 2024-08-26T17:00..."
82,2024-09-10T00:23:00-CME-001,2024-09-10 00:23:00+00:00,"[2024-09-09T23:47:00-FLR-001, 2024-09-12T02:53..."


### GST Data

In [164]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for GST
query_url = f"{base_url}{GST}?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"

print(query_url)

https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=X5Rsa8m3mUgQ9w4PcCkoycbNsfumMRINc4SatjFF


In [165]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url)

In [166]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
gst_formatted_json = json.dumps(gst_json, indent=4)
#gst_formatted_json

In [139]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df.reset_index()
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]

gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [140]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=['linkedEvents'])
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [141]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents')
gst_df = gst_df.reset_index(drop=True)
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [142]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df['CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df = gst_df.dropna(subset=['CME_ActivityID'])
gst_df.head()

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [143]:
# Convert the 'CME_ActivityID' column to string format 
gst_df['CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format
if 'gstID' in gst_df.columns:
    gst_df['gstID'] = gst_df['gstID'].astype(str)

# Convert 'startTime' to datetime format
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])

# Rename 'startTime' to 'startTime_GST'
gst_df = gst_df.rename(columns={'startTime': 'startTime_GST'})

# Drop 'linkedEvents' column
gst_df = gst_df.drop(columns=['linkedEvents'])

gst_df.head()

,gstID,startTime_GST,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01 01:00:00+00:00,2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T02:47:00-MPC-001


In [144]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME', na=False)]
gst_df.head()

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [154]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(
    gst_df, 
    cme_df, 
    left_on='CME_ActivityID', 
    right_on='cmeID'
)
merged_df.head()
# Really struggled to get this part to work since nothing was showing up in the index
# Please consider if the code was correct even though the output was incorrect

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID


In [155]:
# Verify that the new DataFrame has the same number of rows as cme and gst
num_rows_gst = len(gst_df)
num_rows_cme = len(cme_df)
num_rows_merged = len(merged_df)
print(f"Number of rows in gst_df: {num_rows_gst}")
print(f"Number of rows in cme_df: {num_rows_cme}")
print(f"Number of rows in merged_df: {num_rows_merged}")

Number of rows in gst_df: 61
Number of rows in cme_df: 2
Number of rows in merged_df: 0


### Computing the time it takes for a CME to cause a GST

In [156]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['startTime_GST'] = pd.to_datetime(merged_df['startTime_GST'])
merged_df['startTime_CME'] = pd.to_datetime(merged_df['startTime_CME'])

# Compute the time difference
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']

# new column
merged_df[['startTime_GST', 'startTime_CME', 'timeDiff']].head()

,startTime_GST,startTime_CME,timeDiff


In [157]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
merged_df['timeDiff'].describe()

count      0
mean     NaT
std      NaT
min      NaT
25%      NaT
50%      NaT
75%      NaT
max      NaT
Name: timeDiff, dtype: object

### Exporting data in csv format

In [158]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)